In [4]:
import gymnasium as gym
import highway_env

# Agent
from stable_baselines3 import DQN


import sys
from tqdm.notebook import trange

from stable_baselines3 import DQN
import pprint
from matplotlib import pyplot as plt
import numpy as np
import joblib
from tqdm import trange

2025-01-17 09:15:24.803078: I tensorflow/core/util/port.cc:110] oneDNN custom operations are on. You may see slightly different numerical results due to floating-point round-off errors from different computation orders. To turn them off, set the environment variable `TF_ENABLE_ONEDNN_OPTS=0`.
2025-01-17 09:15:24.852987: I tensorflow/tsl/cuda/cudart_stub.cc:28] Could not find cuda drivers on your machine, GPU will not be used.
2025-01-17 09:15:25.268559: I tensorflow/core/platform/cpu_feature_guard.cc:182] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 AVX512F AVX512_VNNI FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.
2025-01-17 09:15:26.578379: W tensorflow/compiler/tf2tensorrt/utils/py_utils.cc:38] TF-TRT Warning: Could not find TensorRT


In [6]:
import base64
from pathlib import Path

from gymnasium.wrappers import RecordVideo
from IPython import display as ipythondisplay
from pyvirtualdisplay import Display



def record_videos(env, video_folder="videos"):
    wrapped = RecordVideo(
        env, video_folder=video_folder, episode_trigger=lambda e: True
    )

    # Capture intermediate frames
    env.unwrapped.set_record_video_wrapper(wrapped)

    return wrapped


def show_videos(path="videos"):
    html = []
    for mp4 in Path(path).glob("*.mp4"):
        video_b64 = base64.b64encode(mp4.read_bytes())
        html.append(
            """<video alt="{}" autoplay
                      loop controls style="height: 400px;">
                      <source src="data:video/mp4;base64,{}" type="video/mp4" />
                 </video>""".format(
                mp4, video_b64.decode("ascii")
            )
        )
    ipythondisplay.display(ipythondisplay.HTML(data="<br>".join(html)))


modify obs

In [7]:
def extract_features_from_dataset(data):
    """
    Extract features from the dataset based on the given criteria.
    """
    processed_data = []

    for row in data:
        # Ego vehicle features
        ego_features = row[:5]
        ego_lane = ego_features[2]//4 + 1  # Lane ID of ego vehicle
        ego_speed = ego_features[3]  # Speed of ego vehicle

        # Other vehicles' features
        other_vehicles = row[5:50].reshape(9, 5)  # 9 vehicles, 5 features each
        # actions = row[50]
        # Separate features of other vehicles
        other_lanes = other_vehicles[:, 2] // 4 + 1  # Lane IDs of other vehicles
        distances = np.abs(other_vehicles[:, 1] - ego_features[1])  # Distances from ego vehicle
        relative_velocities = other_vehicles[:, 3] - ego_speed  # Relative velocities

        # Number of vehicles in ego lane and adjacent lanes
        vehicles_in_ego_lane = np.sum(other_lanes == ego_lane)
        vehicles_in_left_lane = np.sum(other_lanes == ego_lane - 1)
        vehicles_in_right_lane = np.sum(other_lanes == ego_lane + 1)

        ## Closest vehicles
        closest_ego_index = np.where(other_lanes == ego_lane, distances, np.inf).argmin() if vehicles_in_ego_lane !=0 else np.NAN
        closest_left_index = np.where(other_lanes == ego_lane - 1, distances, np.inf).argmin() if vehicles_in_left_lane !=0 else np.NAN
        closest_right_index = np.where(other_lanes == ego_lane + 1, distances, np.inf).argmin() if vehicles_in_right_lane !=0 else np.NAN

        # Distances of other vehicles
        ##ego
        if np.isnan(closest_ego_index):
            closest_in_ego_lane_dist = 10000  # Assign large value for no vehicle
        else:
            closest_in_ego_lane_dist = distances[closest_ego_index]  
        ##left
        if np.isnan(closest_left_index):
            closest_left_lane_dist = 10000  # Assign default value for no vehicle
        else:
            closest_left_lane_dist = distances[closest_left_index]  
        ##right
        if np.isnan(closest_right_index):
            closest_right_lane_dist = 10000  # Assign default value for no vehicle
        else:
            closest_right_lane_dist = distances[closest_right_index]  

        ##relative velocities of closest vehicles
        ##ego
        if np.isnan(closest_ego_index):
            relative_velocity_ego_lane = 10000  # Assign large value for no vehicle
        else:
            relative_velocity_ego_lane = relative_velocities[closest_ego_index]  
        ##left
        if np.isnan(closest_left_index):
            relative_velocity_left_lane = 10000  # Assign large value for no vehicle
        else:
            relative_velocity_left_lane = relative_velocities[closest_left_index]  
        ##right
        if np.isnan(closest_right_index):
            relative_velocity_right_lane = 10000  # Assign large value for no vehicle
        else:
            relative_velocity_right_lane = relative_velocities[closest_right_index]  
        
        # Append computed features
        processed_data.append([
            vehicles_in_ego_lane,
            vehicles_in_left_lane,
            vehicles_in_right_lane,
            closest_in_ego_lane_dist,
            closest_left_lane_dist,
            closest_right_lane_dist,
            relative_velocity_ego_lane,
            relative_velocity_left_lane,
            relative_velocity_right_lane
        ])

    return np.array(processed_data)

In [18]:
def rf_query(obs):
    ##load model
    rf_model = joblib.load("models_try/rf_test")
    #print(f"Model loaded")
    ##
    obs_flat = obs.flatten() # Flatten and reshape observation
    obs_processed = extract_features_from_dataset([obs_flat])[0]
    return rf_model.predict(obs_processed)[0]

In [19]:
import os
import gymnasium as gym
import numpy as np
from gymnasium.wrappers import RecordVideo
from tqdm import trange  # For progress tracking
from IPython.display import display, Video

# Base setting
vehicleCount = 10

# Environment configuration
config = {
    "observation": {
        "type": "Kinematics",
        "features": ["presence", "x", "y", "vx", "vy"],
        "absolute": True,
        "normalize": True,
        "vehicles_count": vehicleCount,
        "see_behind": True,
    },
    "action": {
        "type": "DiscreteMetaAction",
        "target_speeds": np.linspace(0, 32, 9),
    },
    "duration": 40,
    "vehicles_density": 1,
    "show_trajectories": True,
    "render_agent": True,}

# Create directory for video storage if it doesn't exist
video_dir = 'videos/test_safe_Claude'
if not os.path.exists(video_dir):
    os.makedirs(video_dir)

# Create directory for action predictions storage if it doesn't exist
predictions_dir = 'predictions/test_safe_Claude'
if not os.path.exists(predictions_dir):
    os.makedirs(predictions_dir)

# Create and configure environment
env = gym.make('highway-v0', render_mode='rgb_array', config=config)

# Wrap environment with video recording
env = RecordVideo(env, video_folder=video_dir, episode_trigger=lambda episode_id: True)


# Run test episodes and record them
for episode in trange(3, desc='Test episodes'):
    # Prepare to log the actions taken in this episode
    actions_file_path = os.path.join(predictions_dir, f'episode_{episode}_actions.txt')
    
    with open(actions_file_path, 'w') as action_file:
        (obs, info), done, truncated = env.reset(), False, False
        while not (done or truncated):
            #action = 4 if np.random.rand() < 0.5 else 3
            action = rf_query(obs)
            obs, reward, done, truncated, info = env.step(int(action))
            
            # Log the action taken
            action_file.write(f"Action: {action}\n")
env.close()


# Display the recorded videos
show_videos()


/home/prachit/.local/lib/python3.8/site-packages/gymnasium/wrappers/monitoring/video_recorder.py:178: UserWarning: WARN: Unable to save last video! Did you call close()?
  logger.warn("Unable to save last video! Did you call close()?")
/home/prachit/.local/lib/python3.8/site-packages/gymnasium/wrappers/record_video.py:94: UserWarning: WARN: Overwriting existing videos at /home/prachit/Desktop/Reward-shaping-with-LLMS/videos/test_safe_Claude folder (try specifying a different `video_folder` for the `RecordVideo` wrapper if this is not desired)
  logger.warn(
Test episodes:   0%|          | 0/3 [00:00<?, ?it/s]/home/prachit/.local/lib/python3.8/site-packages/sklearn/base.py:465: UserWarning: X does not have valid feature names, but RandomForestClassifier was fitted with feature names
  warnings.warn(
Test episodes:   0%|          | 0/3 [00:00<?, ?it/s]


ValueError: Expected 2D array, got 1D array instead:
array=[ 9.0000000e+00  0.0000000e+00  0.0000000e+00  9.6991897e-02
  1.0000000e+04  1.0000000e+04 -1.9715577e-02  1.0000000e+04
  1.0000000e+04].
Reshape your data either using array.reshape(-1, 1) if your data has a single feature or array.reshape(1, -1) if it contains a single sample.